<a href="https://colab.research.google.com/github/HariBejju/fine-tuning-basics/blob/main/finetuning_kanna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers


In [2]:
pip install torch


In [3]:
pip install --upgrade transformers

In [6]:
from google.colab import userdata
hf_token=userdata.get('hf_token')

In [7]:
# Install required libraries


# Import necessary libraries
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import login
import torch

# Login to Hugging Face (replace with your token)
login(hf_token)

# Set the model id
model_id = "meta-llama/Llama-3.2-1B-Instruct"

# Check if CUDA is available and set the device accordingly
if torch.cuda.is_available():
    device = 'cuda'
    print("CUDA is available. Using GPU.")
    print(torch.cuda.get_device_name(0))
    model = AutoModelForCausalLM.from_pretrained(model_id).to(device) #instantiate model to GPU

else:
    device = 'cpu'
    print("CUDA is not available. Using CPU.")
    model = AutoModelForCausalLM.from_pretrained(model_id) #instantiate model to CPU

#Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

# set pad token.
tokenizer.pad_token = tokenizer.eos_token # Set the pad token to be the end of sequence token


# Create the pipeline
generating_pipeline = pipeline(task='text-generation', model=model, tokenizer=tokenizer, device=device)

# Test the pipeline
print(generating_pipeline("vanakam bro", max_new_tokens=25))



CUDA is not available. Using CPU.


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Device set to use cpu


[{'generated_text': "vanakam bro, i'm excited to share with you my latest purchase. I just got my hands on the new Samsung Galaxy Z Fold3"}]


In [8]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding=True, truncation=True)


In [9]:
!pip install --upgrade transformers datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [10]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving financial_dataset_100_companies.txt to financial_dataset_100_companies.txt
User uploaded file "financial_dataset_100_companies.txt" with length 22019 bytes


In [11]:
data_path = '/content/financial_dataset_100_companies.txt'
with open(data_path, 'r') as file:
    lines = file.readlines()
print(lines[:5])

["Company 1 has an annual revenue of 5100 crores and a net profit of -505 crores. The company's debt-to-equity ratio is 0.6, with an EPS of -10.3. It has a market capitalization of 10200 crores and is considered Unstable.\n", "Company 2 has an annual revenue of 5200 crores and a net profit of 520 crores. The company's debt-to-equity ratio is 0.7, with an EPS of -10.6. It has a market capitalization of 10400 crores and is considered Unstable.\n", "Company 3 has an annual revenue of 5300 crores and a net profit of -515 crores. The company's debt-to-equity ratio is 0.8, with an EPS of 21.5. It has a market capitalization of 10600 crores and is considered Stable.\n", "Company 4 has an annual revenue of 5400 crores and a net profit of 540 crores. The company's debt-to-equity ratio is 0.9, with an EPS of -11.2. It has a market capitalization of 10800 crores and is considered Unstable.\n", "Company 5 has an annual revenue of 5500 crores and a net profit of -525 crores. The company's debt-to-e

In [12]:
!pip install datasets

In [13]:
from datasets import Dataset

# Create a Dataset object
dataset = Dataset.from_dict({"text": lines})

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], padding=True, truncation=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [14]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./fine_tuned_model",  # Where to save the model
    overwrite_output_dir=True,
    num_train_epochs=3,  # Number of epochs
    per_device_train_batch_size=8,  # Batch size
    save_steps=500,  # Save the model every 500 steps
    save_total_limit=2,  # Keep only the last 2 models
    logging_dir="./logs",  # Logging directory
    logging_steps=10,  # Log every 10 steps
    evaluation_strategy="no",  # Evaluate during training
    eval_steps=200,  # Evaluation frequency
    fp16=True if device == 'cuda' else False,  # Mixed precision for faster training
    learning_rate=5e-5,  # Learning rate
    report_to="none"  # Avoid unwanted reporting
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [15]:
from transformers import Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer
)

<ipython-input-15-df7f49cf8ad5>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
